In [2]:
#uses pandas and then convert to list/dict
import pandas as pd 
import string
from string import punctuation
import re
import emoji
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
import torch.nn as nn
import torch
from sklearn.utils import shuffle
from torch.nn import functional as F
import matplotlib.pyplot as plt
import scikitplot as skplt
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import itertools
import time
import contractions
import pickle


import os
import numpy as np

In [3]:
data_folder = 'data/'
#glove_folder = 'E:/rajiur/workspace/nlp/data/'
glove_folder = 'D:/workspace/nlp/glove/'

In [8]:
dfdevraw = pd.read_csv(data_folder+"dev.csv", encoding='latin') 
dftrainraw =  pd.read_csv(data_folder+"train.csv", encoding='latin') 

In [9]:
dfdevraw.head()

,ID,Comment,Label
0,facebook_corpus_msr_451811,The quality of re made now makes me think it i...,CAG
1,facebook_corpus_msr_334368,@siva \rHow is ur mother???\rHow is ur wife???...,NAG
2,facebook_corpus_msr_331195,Also see ....hw ur RSS activist caught in Burk...,NAG
3,facebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING\r...,NAG
4,facebook_corpus_msr_379239,Modi ho ya Manmohan singh saala yeh log kuch n...,OAG


In [6]:
dftrainraw.head()

,ID,Comment,Label
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG


**Pre-Processing**

In [10]:
def cleanup_text(texts):
    cleaned_text = []
    for text in texts:
        # remove ugly &quot and &amp
        text = re.sub(r'&quot;(.*?)&quot;', "\g<1>", text)
        text = re.sub(r'&amp;', "", text)

        # replace emoticon
        text = re.sub(r'(^| )(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)', "\g<1>TOKEMOTICON", text)

        text = text.lower()
        text = text.replace("tokemoticon", "TOKEMOTICON")

        # replace url
        text = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',
                    "TOKURL", text)

        # replace mention
        text = re.sub(r'@[\w]+', "TOKMENTION", text)

        # replace hashtag
        text = re.sub(r'#[\w]+', "TOKHASHTAG", text)

        # replace dollar
        text = re.sub(r'\$\d+', "TOKDOLLAR", text)

        # remove punctuation
        text = re.sub('[^a-zA-Z0-9]', ' ', text)

        # remove multiple spaces
        text = re.sub(r' +', ' ', text)

        # remove newline
        text = re.sub(r'\n', ' ', text)

        cleaned_text.append(text)
    return cleaned_text

In [11]:
#def removeurl(string):
#    return re.sub(r'^http[s]?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)

class2index = {'OAG':2,'CAG':0,'NAG':1}
index2class = {2:'OAG',0:'CAG',1:'NAG'}


def removeurl(sample):
    """Remove URLs from a sample string"""
    #return re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", sample.lower())
    return re.sub(r"http\S+", "", sample)

def removenonascii(post):
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, post))

def removenewline(string):
    string = string.replace('\r', ' ')
    string = string.replace('\n', ' ')
    return string    

def removepunctuation(string):
    return ''.join([c for c in string if c not in punctuation])

def removepunctuationwithspace(string):
    string = re.sub(r'[^\w\s]',' ',string)
    return string

def removepunctuationandnumbers(string):
    string = re.sub('[^a-zA-Z]', ' ', string)
    return string


def stemwords(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatizeverbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def rawprocess(df):
    result = pd.DataFrame(df)
    countpunctuation = []
    countexclamation = []
    
    for index, row in result.iterrows():
        countpunctuation.append(len([c for c in row['Comment'] if c in punctuation]))
        countexclamation.append(len([c for c in row['Comment'] if c in ['!','?']]))
    result['countpunctuation'] = countpunctuation 
    result['countexclamation'] = countexclamation    
    return result    
  


"""def preprocess_old(df):
    result = pd.DataFrame(df)
    labelnumber = []
    for index, row in result.iterrows():
        #print(row['c1'], row['c2'])
        row['Comment'] = removepunctuation(removenewline(removenonascii(emoji.demojize(removeurl(row['Comment']))))).lower()
        labelnumber.append(class2index[row['Label']])
        #if(row['Comment']==''):
        #    print(row['ID']   
    result['labelnumber'] = labelnumber    
    return result"""    

def preprocess(df, haslabel=True):
    result = pd.DataFrame(df)
    labelnumber = []
    commentnotlowered = []
    for index, row in result.iterrows():
        #print(row['c1'], row['c2'])
        
        row['Comment'] = contractions.fix(row['Comment'])
        comment = removepunctuationandnumbers(removenewline(removenonascii(emoji.demojize(removeurl(row['Comment'])))))
        commentnotlowered.append(comment)
        row['Comment'] = comment.lower()
        if haslabel:
            labelnumber.append(class2index[row['Label']])
        #if(row['Comment']==''):
        #    print(row['ID']   
    if haslabel:
        result['labelnumber'] = labelnumber    
    result['commentnotlowered'] = commentnotlowered
    return result


def tokenization_old(df):
    alltokens = []
    tokenizer = TweetTokenizer()
    for index, row in df.iterrows():
        commenttokens = tokenizer.tokenize(row['Comment'])
        alltokens.append(commenttokens)
        
    df['tokens'] = alltokens    
    return df


def tokenization(df):
    result = pd.DataFrame(df)
    alltokens = []
    alltokensnotlowered = []
    tokenizer = TweetTokenizer()
    for index, row in df.iterrows():
        commenttokens = tokenizer.tokenize(row['commentnotlowered'])
        
        alltokensnotlowered.append(commenttokens)
        alltokens.append([x.lower() for x in commenttokens])
        
    result['tokens'] = alltokens 
    result['tokensnotlowered'] = alltokensnotlowered 
    
    return result

In [33]:
dftrain = preprocess(dftrainraw)
dfdev = preprocess(dfdevraw)

In [36]:
# tokenization
dfdev = tokenization(dfdev)
dftrain  = tokenization(dftrain)
train_tokens = [row['tokens'] for index,row in dftrain.iterrows()]
dev_tokens = [row['tokens'] for index,row in dfdev.iterrows()]



In [34]:
dftrain.head()

,ID,Comment,Label,labelnumber,commentnotlowered,tokens,tokensnotlowered
0,facebook_corpus_msr_1723796,well said sonu you have courage to stand agai...,OAG,2,well said sonu you have courage to stand agai...,"[well, said, sonu, you, have, courage, to, sta...","[Well, said, sonu, you, have, courage, to, sta..."
1,facebook_corpus_msr_466073,most of private banks atm s like hdfc icici e...,NAG,1,most of private banks atm s like hdfc icici e...,"[most, of, private, banks, atm, s, like, hdfc,...","[Most, of, Private, Banks, ATM, s, Like, HDFC,..."
2,facebook_corpus_msr_1493901,now question is pakistan will adhere to this,OAG,2,now question is pakistan will adhere to this,"[now, question, is, pakistan, will, adhere, to...","[Now, question, is, Pakistan, will, adhere, to..."
3,facebook_corpus_msr_405512,pakistan is comprised of fake muslims who does...,OAG,2,pakistan is comprised of fake muslims who does...,"[pakistan, is, comprised, of, fake, muslims, w...","[Pakistan, is, comprised, of, fake, muslims, w..."
4,facebook_corpus_msr_1521685,we r against cow slaughter so of course it w...,NAG,1,we r against cow slaughter so of course it w...,"[we, r, against, cow, slaughter, so, of, cours...","[we, r, against, cow, slaughter, so, of, cours..."


In [38]:
dfdev['tokens'][0],dfdev['tokens'][1]

(['the',
  'quality',
  'of',
  're',
  'made',
  'now',
  'makes',
  'me',
  'think',
  'it',
  'is',
  'something',
  'to',
  'be',
  'bought',
  'from',
  'fish',
  'market'],
 ['siva',
  'how',
  'is',
  'ur',
  'mother',
  'how',
  'is',
  'ur',
  'wife',
  'how',
  'is',
  'ur',
  'sister',
  'hope',
  'everyone',
  'is',
  'fine'])

In [19]:
dfdev.head()

,ID,Comment,Label,labelnumber,commentnotlowered,tokens,tokensnotlowered
0,facebook_corpus_msr_451811,the quality of re made now makes me think it i...,CAG,0,The quality of re made now makes me think it i...,"[the, quality, of, re, made, now, makes, me, t...","[The, quality, of, re, made, now, makes, me, t..."
1,facebook_corpus_msr_334368,siva how is ur mother how is ur wife h...,NAG,1,siva How is ur mother How is ur wife H...,"[siva, how, is, ur, mother, how, is, ur, wife,...","[siva, How, is, ur, mother, How, is, ur, wife,..."
2,facebook_corpus_msr_331195,also see hw ur rss activist caught in burk...,NAG,1,Also see hw ur RSS activist caught in Burk...,"[also, see, hw, ur, rss, activist, caught, in,...","[Also, see, hw, ur, RSS, activist, caught, in,..."
3,facebook_corpus_msr_403402,on the death of 2 jawans in loc cross firing ...,NAG,1,On the death of 2 jawans in LOC CROSS FIRING ...,"[on, the, death, of, 2, jawans, in, loc, cross...","[On, the, death, of, 2, jawans, in, LOC, CROSS..."
4,facebook_corpus_msr_379239,modi ho ya manmohan singh saala yeh log kuch n...,OAG,2,Modi ho ya Manmohan singh saala yeh log kuch n...,"[modi, ho, ya, manmohan, singh, saala, yeh, lo...","[Modi, ho, ya, Manmohan, singh, saala, yeh, lo..."


In [39]:
dfdev.to_pickle('dev_tokenized_df.pkl')
dftrain.to_pickle('train_tokenized_df.pkl')

In [40]:
# Create a dictionary of tokens
vocabulary = set([t for tokens in train_tokens + dev_tokens for t in tokens])
print('Vocabulary size: {}'.format(len(vocabulary)))

Vocabulary size: 22138


In [41]:
import pickle
with open('vocabulary.pkl', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(vocabulary, filehandle)

In [52]:
dftestraw =  pd.read_csv("data/test_no_label.csv", encoding='latin') 


In [53]:
dftest = preprocess(dftestraw, False)
dftest  = tokenization(dftest)

In [54]:
dftest.head()

,ID,Comment,commentnotlowered,tokens,tokensnotlowered
0,facebook_corpus_msr_495558,but if same had been given before olymic games...,But if same had been given before olymic games...,"[but, if, same, had, been, given, before, olym...","[But, if, same, had, been, given, before, olym..."
1,facebook_corpus_msr_1561809,does our constitution gives power to misbehave...,Does our constitution gives power to misbehave...,"[does, our, constitution, gives, power, to, mi...","[Does, our, constitution, gives, power, to, mi..."
2,facebook_corpus_msr_442487,automotive awards are done at the cost of spar...,automotive awards are done at the cost of spar...,"[automotive, awards, are, done, at, the, cost,...","[automotive, awards, are, done, at, the, cost,..."
3,facebook_corpus_msr_495517,i appreciate the stated facts but the last one...,I appreciate the stated facts but the last one...,"[i, appreciate, the, stated, facts, but, the, ...","[I, appreciate, the, stated, facts, but, the, ..."
4,facebook_corpus_msr_1805455,rss agent mr hazare how you are feeling now,RSS agent Mr Hazare how you are feeling now,"[rss, agent, mr, hazare, how, you, are, feelin...","[RSS, agent, Mr, Hazare, how, you, are, feelin..."


In [55]:
dftest.to_pickle('test_tokenized_df.pkl')

**Word Embedding**

In [12]:


class WordEmbedder(nn.Module):
    def __init__(self, vocab, glove_file, seqlen = 300, varlen = False):
        super(WordEmbedder, self).__init__()
        assert os.path.exists(glove_file) and glove_file.endswith('.txt'), glove_file
        
        self.emb_dim = None
        
        self.PAD_TOKEN = '<PAD>'
        self.UNK_TOKEN = '<UNK>'
        self.sequence_length = seqlen
        self.various_length = varlen
        
        idx2word = [self.PAD_TOKEN, self.UNK_TOKEN]
        idx2vect = [None, None]
        
        with open(glove_file, 'r', encoding='utf-8') as fp:
            for line in fp:
                line = line.split()
                
                if line[0] not in vocab:
                    continue
                
                w = line[0]
                v = np.array([float(value) for value in line[1:]])
                
                if self.emb_dim is None:
                    self.emb_dim = v.shape[0]
            
                idx2word.append(w)
                idx2vect.append(v)
                
        idx2vect[0] = np.zeros(self.emb_dim)
        idx2vect[1] = np.mean(idx2vect[2:], axis=0)
    
        self.embeddings = torch.from_numpy(np.array(idx2vect)).float()
        self.embeddings = nn.Embedding.from_pretrained(self.embeddings, freeze=False)
        
        self.idx2word = {i: w for i, w in enumerate(idx2word)}
        self.word2idx = {w: i for i, w in self.idx2word.items()}
    
    def forward(self, samples):
        pad_idx = self.word2idx[self.PAD_TOKEN]
        unk_idx = self.word2idx[self.UNK_TOKEN]
        
        if self.various_length:

            #Find the length of the longest sample
            maxlen = max([len(s) for s in samples])
        
        else:

            #Use a constant length for all samples
            maxlen = self.sequence_length
        
        encoded = [[self.word2idx.get(token, unk_idx) for token in tokens] for tokens in samples]
        
        padded = np.zeros((len(samples), maxlen), dtype=int)
        masks = torch.zeros(len(samples), maxlen).long()
        
        # Padding and masking
        for i in range(len(encoded)):
            masks[i, :len(encoded[i])] = 1
            padded[i, :len(encoded[i])] = np.array(encoded[i])[:maxlen]
            # encoded[i] += [pad_idx] * max(0, (maxlen - len(encoded[i])))
        
        encoded = torch.tensor(padded).long()
        
        if torch.cuda.is_available():
            encoded = encoded.cuda()
            masks = masks.cuda()
        
        result = {
            'output': self.embeddings(encoded),
            'mask': masks,
            'encoded': encoded
        }
        
        return result

In [13]:
embedder = WordEmbedder(vocabulary, glove_folder+'glove.42B.300d.txt', varlen=True)
embedder

NameError: name 'vocabulary' is not defined

In [51]:
torch.save(embedder,'embedder_glove.pt')

c:\python37\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type WordEmbedder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Hand-crafted Features - Comment level**

In [ ]:
def handcraftedfeatures(df):
    pf= ProfanityFilter()
    
    result = pd.DataFrame(df)
    
    negativetokencount = []
    titletokencount = []
    allcaptokencount = []
    
    for index,row in result.iterrows():
        commenttokens = row['tokensnotlowered']
        #print(row['commentnotlowered'])
        negativecount =0
        titlecount = 0
        allcapcount = 0
        
        for token in commenttokens:
           # print(token,end='')
            if token.istitle():
            #    print(" "+str(titlecount),end='')
                titlecount=titlecount+1
            #    print(" title ("+str(titlecount)+")",end='')
            if token.isupper():
                allcapcount+=1
            if not pf.is_clean(token.lower()):
                negativecount+=1
            #print("")
                
        negativetokencount.append(negativecount)
        titletokencount.append(titlecount)
        allcaptokencount.append(allcapcount)
        
        if(index%100==0):
            print("Completed",index)
 
    result['negativetokencount'] = negativetokencount  
    result['titletokencount'] = titletokencount
    result['allcaptokencount'] = allcaptokencount
    
    return result



def handcraftedfeatures2(df):
    en_dict = enchant.Dict("en_US")
    
    result = pd.DataFrame(df)
    
    totaltokencount = []
    nonenglishtokencount = []
    
    
    for index,row in result.iterrows():
        commenttokens = row['tokens']
        #print(row['commentnotlowered'])
        totaltokens = len(commenttokens)
        
        nonenglishtokens = sum([ 0 if en_dict.check(token) else 1 for token in commenttokens])
           
                
        totaltokencount.append(totaltokens)
        nonenglishtokencount.append(nonenglishtokens)
        
        if(index%100==0):
            print("Completed",index)
 
    result['totaltokencount'] = totaltokencount  
    result['nonenglishtokencount'] = nonenglishtokencount
    
    return result

In [ ]:
start = time.time()
handtrain = handcraftedfeatures(dftrain)
handtrain = handcraftedfeatures2(handtrain)
end = time.time()
print("train completed",time.strftime("%H:%M:%S", time.gmtime(end-start)))
handtrain.to_pickle('train_tokenized_handfeat_df.pkl')

In [ ]:

start = time.time()
handdev = handcraftedfeatures(dfdev)
handdev = handcraftedfeatures2(handdev)
end = time.time()
print("dev completed",time.strftime("%H:%M:%S", time.gmtime(end-start)))
handdev.to_pickle('dev_tokenized_handfeat_df.pkl')

**Hand-crafted features - Word level**

In [113]:
def handfeatureextract(samples, sequence_length=300,various_length=True):
    

    maxlen = 0
    
    if various_length:
        #Find the length of the longest sample
        #maxlen = max([len(row['tokensnotlowered']) for index,row in df.iterrows()])
        maxlen = max([len(s) for s in samples])
    else:
        #Use a constant length for all samples
        maxlen = sequence_length
    
    basicfeatures = []
    for tokens in samples:
        #comment = row['commentnotlowered']
        
        taggedtokens = nltk.pos_tag(' '.join(tokens))
        
        #print(tokens, taggedtokens)
        commentbasicfeatures = addfeatures(tokens, taggedtokens, maxlen)
        
        #print(len(commentbasicfeatures))
        
        basicfeatures.append(commentbasicfeatures)
        
        handfeatures = [[[word[key] for key in word] for word in handfeature] for handfeature in basicfeatures]
    
    return handfeatures
    